# Creating a frequency finder auto-encoder

## Import Libraries

In [11]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from helpers import *
import os
import getpass
import subprocess

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get drive path 

In [13]:
drive_path = find_drive_path()
drive_path

NameError: name 'find_drive_path' is not defined